# General Number Field Sieve - Interactive Notebook

This notebook lets you run the GNFS algorithm on your own hardware.

## Setup

First, install the required packages:

In [ ]:
# Run this cell first to install dependencies
!pip install sympy numpy

## GNFS Implementation

Clone the repository and import the GNFS module:

In [ ]:
# Clone the repository (run once)
!git clone https://github.com/toddsmithie/General-Number-Field-Sieve.git

# Add to path
import sys
sys.path.insert(0, './General-Number-Field-Sieve')

## Run GNFS Factorization

Configure parameters and factor your number:

In [ ]:
import time
import sympy as sp
from gnfs import select_polynomial, find_relations, find_factors

# ===== CONFIGURATION =====
n = 91                # Integer to factor
degree = 1            # Polynomial degree (1 for small numbers, higher for large)
bound = 20            # Smoothness bound (max prime in factor base)
interval = 50         # Sieve interval (search range)
# =========================

start = time.time()

print(f"GNFS Factorization of n = {n}")
print("=" * 60)

# Step 0: Primality check (polynomial time using Miller-Rabin)
print("\nSTEP 0: PRIMALITY CHECK")
print("-" * 60)
if sp.isprime(n):
    elapsed = time.time() - start
    print("Number is PRIME - cannot be factored!")
    print(f"\n{'='*60}")
    print(f"RESULT: {n} is prime")
    print(f"{'='*60}")
    print(f"Time: {elapsed:.3f}s")
else:
    print(f"{n} is composite - proceeding with factorization...")
    
    # Step 1: Polynomial selection
    print("\nSTEP 1: POLYNOMIAL SELECTION")
    print("-" * 60)
    selection = select_polynomial(n, degree=degree)
    print(f"Choose m = floor(n^(1/d)) = {selection.m}")
    print(f"Algebraic polynomial f(x) = {selection.algebraic}")
    print(f"Rational polynomial g(x) = {selection.rational}")
    print(f"These share root m = {selection.m} modulo n = {n}")
    
    # Step 2: Build factor base and sieve
    print("\nSTEP 2: SIEVING FOR SMOOTH RELATIONS")
    print("-" * 60)
    primes = list(sp.primerange(2, bound + 1))
    print(f"Factor base (primes <= {bound}): {primes}")
    print(f"Searching for relations in interval [-{interval}, {interval}]...")
    
    relations = list(find_relations(selection, primes=primes, interval=interval))
    print(f"Found {len(relations)} smooth relations")
    
    if relations:
        print(f"\nRelations (a, b) where a - {selection.m}*b factors over the base:")
        shown = min(6, len(relations))
        for i, rel in enumerate(relations[:shown]):
            val = rel.a - selection.m * rel.b
            factors_str = " * ".join(
                f"{p}^{e}" if e > 1 else str(p) 
                for p, e in rel.rational_factors.items()
            ) if rel.rational_factors else "1"
            print(f"  ({rel.a:4}, {rel.b:2}): {rel.a:4} - {selection.m}*{rel.b:2} = {val:5} = {factors_str}")
        if len(relations) > shown:
            print(f"  ... ({len(relations) - shown} more relations)")
    
    # Step 3: Linear algebra
    print("\nSTEP 3: LINEAR ALGEBRA OVER GF(2)")
    print("-" * 60)
    print(f"Building exponent matrix ({len(relations)} relations x {len(primes)} primes)...")
    print("Finding vectors in nullspace (dependencies with even exponents)...")
    
    # Step 4: Extract factors
    print("\nSTEP 4: SQUARE ROOT AND FACTOR EXTRACTION")
    print("-" * 60)
    
    factors = list(find_factors(n, relations, primes))
    elapsed = time.time() - start
    
    if factors and len(factors) >= 2:
        p, q = factors[0], factors[1]
        print("Combining relations to form congruence of squares...")
        print("Found: x^2 = y^2 (mod n) where x != +/-y")
        print("gcd(x - y, n) gives non-trivial factor!")
        print()
        print("=" * 60)
        print(f"RESULT: {n} = {p} x {q}")
        print("=" * 60)
        print(f"Verification: {p} x {q} = {p * q}")
        print(f"Time: {elapsed:.3f}s")
    else:
        print("No factors found with current parameters.")
        print()
        print("Tips:")
        print("  - Try a smaller semiprime (91, 143 work well)")
        print("  - Increase smoothness bound")
        print("  - Increase sieve interval")

## Experiment with Different Numbers

Try these examples or your own:

In [ ]:
# Example: Small semiprimes that work well
test_numbers = [
    (91, "7 × 13"),
    (143, "11 × 13"),
    (221, "13 × 17"),
    (323, "17 × 19"),
]

for num, note in test_numbers:
    print(f"\n{'='*60}")
    print(f"Testing n = {num} ({note})")
    print(f"{'='*60}")
    # Copy the factorization code from above with n = num

## Learn More

- **Repository:** https://github.com/toddsmithie/General-Number-Field-Sieve
- **Documentation:** See the `book/` directory in the repository
- **Algorithm Details:** https://en.wikipedia.org/wiki/General_number_field_sieve